In [1]:
from pymongo import MongoClient
import os
from pprint import pprint

In [2]:
mongo_username = "root"
mongo_password = os.environ.get("PW")
host = "localhost"
port = "27020"  # prod

mongoclient = MongoClient(
    "mongodb://%s:%s@%s:%s" % (mongo_username, mongo_password, host, port)
)

In [2]:
host = "localhost"
port = "27017"  # local

mongoclient_local = MongoClient("mongodb://%s:%s" % (host, port))

In [3]:
mongoclient_local["hazarddb"]["TornadoDataset"].find()

In [5]:
for document in mongoclient_local["hazarddb"]["TornadoDataset"].find():
    document["hazardDatasets"] = [
        {
            "datasetId": document["datasetId"],
            "demandType": "wind",
            "demandUnits": "mph",
            "threshold": None,
        }
    ]
    # replace document
    mongoclient_local["hazarddb"]["TornadoDataset"].replace_one(
        {"_id": document["_id"]}, document
    )
    # pprint(document)

In [ ]:
## run after we have updated all other components and we are ready to remove this key.

for document in mongoclient_local["hazarddb"]["TornadoDataset"].find():
    # remove datasetId key from main document
    del document["datasetId"]
    # replace document
    mongoclient_local["hazarddb"]["TornadoDataset"].replace_one(
        {"_id": document["_id"]}, document
    )